In [1]:
import pandas as pd

In [2]:
def create_df(filename):
    # Read the file into a list of lines
    with open(filename, 'r') as file:
        lines = file.readlines()

    # Process the table header, given in line 4
    table_header = lines[4].split(',')
    table_header = [s.strip().replace('\n', '') for s in table_header]

    # Initialize an empty list to store the data
    data = []

    # Process each line
    for row in lines[6:]:
        # Split the line into fields based on whitespace
        fields = row.split()
        try: 
            fields.remove('//')
        except:
            pass
        
        # Append the fields as a row to the data list
        data.append(fields)

     #Insert table_header at position 0 in data
    data.insert(0, table_header)

    # Create a pandas data frame from the data list
    df = pd.DataFrame(data)
    df.rename(columns=df.iloc[0], inplace=True)
    df.columns = df.columns.str.replace(' ', '_')
    df = df[1:]

    return df


In [3]:
data_df = create_df('INDEX_general_PL_data.2020')
# Drop all columns for which the column name is None
data_df = data_df.loc[:, data_df.columns.notnull()]
data_df.head()

,#_PDB_code,resolution,release_year,-logKd/Ki,Kd/Ki,reference,ligand_name
1,3zzf,2.20,2012,0.40,Ki=400mM,3zzf.pdf,(NLG)
2,3gww,2.46,2009,0.45,IC50=355mM,3gwu.pdf,(SFX)
3,1w8l,1.80,2004,0.49,Ki=320mM,1w8l.pdf,(1P3)
4,3fqa,2.35,2009,0.49,IC50=320mM,3fq7.pdf,(GAB&PMP)
5,1zsb,2.00,1996,0.60,Kd=250mM,1zsb.pdf,(AZM)


In [4]:
name_df = create_df('INDEX_general_PL_name.2020')

# Now column_3 contains the combined strings from columns 3 to x, ignoring None values
name_df['protein_name'] = name_df.iloc[:, 3:].apply(lambda row: ' '.join(str(x) for x in row if x is not None), axis=1)

# Drop all columns for which the column name is None
name_df = name_df.loc[:, name_df.columns.notnull()]

name_df.head()

,#_PDB_code,release_year,Uniprot_ID,protein_name
1,6mu1,2018,P29994,"INOSITOL 1,4,5-TRISPHOSPHATE RECEPTOR TYPE 1"
2,3t8s,2011,P29994,"INOSITOL 1,4,5-TRISPHOSPHATE RECEPTOR TYPE 1"
3,1n4k,2002,P11881,"INOSITOL 1,4,5-TRISPHOSPHATE RECEPTOR TYPE 1"
4,5urm,2017,O75643,U5 SMALL NUCLEAR RIBONUCLEOPROTEIN 200 KDA HEL...
5,5urj,2017,O75643,U5 SMALL NUCLEAR RIBONUCLEOPROTEIN 200 KDA HEL...


In [5]:
df = create_df('INDEX_general_PL.2020')

# Now column_3 contains the combined strings from columns 5 to x, ignoring None values
df['comment'] = df.iloc[:, 6:].apply(lambda row: ' '.join(str(x) for x in row if x is not None), axis=1)

# Drop all columns for which the column name is None
df = df.loc[:, df.columns.notnull()]

df.head()

,#_PDB_code,resolution,release_year,binding_data,reference,ligand_name,comment
1,2tpi,2.10,1982,Kd=49uM,2tpi.pdf,(2-mer),
2,5tln,2.30,1982,Ki=0.43uM,5tln.pdf,(BAN),incomplete ligand structure
3,4tln,2.30,1982,Ki=190uM,4tln.pdf,(LNO),
4,4cts,2.90,1984,Kd<10uM,4cts.pdf,(OAA),
5,6rsa,NMR,1986,Ki=40uM,6rsa.pdf,(UVC),


In [6]:
PDBbind_df = pd.merge(df, name_df, on=['#_PDB_code', 'release_year'])
PDBbind_df.head()

,#_PDB_code,resolution,release_year,binding_data,reference,ligand_name,comment,Uniprot_ID,protein_name
0,2tpi,2.10,1982,Kd=49uM,2tpi.pdf,(2-mer),,P00760,TRYPSINOGEN
1,5tln,2.30,1982,Ki=0.43uM,5tln.pdf,(BAN),incomplete ligand structure,P00800,THERMOLYSIN
2,4tln,2.30,1982,Ki=190uM,4tln.pdf,(LNO),,P00800,THERMOLYSIN
3,4cts,2.90,1984,Kd<10uM,4cts.pdf,(OAA),,P00889,CITRATE SYNTHASE
4,6rsa,NMR,1986,Ki=40uM,6rsa.pdf,(UVC),,P61823,RIBONUCLEASE A


In [7]:
PDBbind_df = pd.merge(df, name_df, on=['#_PDB_code', 'release_year'])
PDBbind_df = pd.merge(PDBbind_df, data_df, on=['#_PDB_code', 'resolution', 'release_year', 'reference', 'ligand_name'])
PDBbind_df.head()

,#_PDB_code,resolution,release_year,binding_data,reference,ligand_name,comment,Uniprot_ID,protein_name,-logKd/Ki,Kd/Ki
0,2tpi,2.10,1982,Kd=49uM,2tpi.pdf,(2-mer),,P00760,TRYPSINOGEN,4.31,Kd=49uM
1,5tln,2.30,1982,Ki=0.43uM,5tln.pdf,(BAN),incomplete ligand structure,P00800,THERMOLYSIN,6.37,Ki=0.43uM
2,4tln,2.30,1982,Ki=190uM,4tln.pdf,(LNO),,P00800,THERMOLYSIN,3.72,Ki=190uM
3,4cts,2.90,1984,Kd<10uM,4cts.pdf,(OAA),,P00889,CITRATE SYNTHASE,5.00,Kd<10uM
4,6rsa,NMR,1986,Ki=40uM,6rsa.pdf,(UVC),,P61823,RIBONUCLEASE A,4.40,Ki=40uM


In [8]:
#duplicates = PDBbind_df['# PDB code'].value_counts() > 1
#print(duplicates[duplicates].index)

In [9]:
# Check if the contents of the 'binding data' and 'Kd/Ki' columns are identical
if (PDBbind_df['binding_data'] == PDBbind_df['Kd/Ki']).all().all():
    # If yes, drop the 'binding data' column
    PDBbind_df = PDBbind_df.drop('binding_data', axis=1)

PDBbind_df.rename(columns={'#_PDB_code':'PDB_ID'}, inplace=True)
PDBbind_df.drop(columns=['reference'], inplace=True)

PDBbind_df.head()

,PDB_ID,resolution,release_year,ligand_name,comment,Uniprot_ID,protein_name,-logKd/Ki,Kd/Ki
0,2tpi,2.10,1982,(2-mer),,P00760,TRYPSINOGEN,4.31,Kd=49uM
1,5tln,2.30,1982,(BAN),incomplete ligand structure,P00800,THERMOLYSIN,6.37,Ki=0.43uM
2,4tln,2.30,1982,(LNO),,P00800,THERMOLYSIN,3.72,Ki=190uM
3,4cts,2.90,1984,(OAA),,P00889,CITRATE SYNTHASE,5.00,Kd<10uM
4,6rsa,NMR,1986,(UVC),,P61823,RIBONUCLEASE A,4.40,Ki=40uM


In [10]:
# remove any '(' or ')' from the strings in the 'ligand_name' column
PDBbind_df['ligand_name'] = PDBbind_df['ligand_name'].str.replace(r'[\(\)]', '', regex=True)

# convert all strings in the 'PDB_ID' column to uppercase
PDBbind_df['PDB_ID'] = PDBbind_df['PDB_ID'].str.upper()

In [11]:
PDBbind_df.head()

,PDB_ID,resolution,release_year,ligand_name,comment,Uniprot_ID,protein_name,-logKd/Ki,Kd/Ki
0,2TPI,2.10,1982,2-mer,,P00760,TRYPSINOGEN,4.31,Kd=49uM
1,5TLN,2.30,1982,BAN,incomplete ligand structure,P00800,THERMOLYSIN,6.37,Ki=0.43uM
2,4TLN,2.30,1982,LNO,,P00800,THERMOLYSIN,3.72,Ki=190uM
3,4CTS,2.90,1984,OAA,,P00889,CITRATE SYNTHASE,5.00,Kd<10uM
4,6RSA,NMR,1986,UVC,,P61823,RIBONUCLEASE A,4.40,Ki=40uM


In [12]:
# split the 'Kd/Ki' column into four parts, based on which type the characters are 
# as 'IC50' comprises letters and numbers we cheat here a bit, and convert the substring to the 'IC' first, and revert that step after the splitting
PDBbind_df['Kd/Ki'] = PDBbind_df['Kd/Ki'].str.replace('IC50', 'IC')
PDBbind_df[['Kd_or_Ki_or_IC50', 'relation', 'value', 'unit']] = PDBbind_df['Kd/Ki'].str.extract(r'([A-Za-z]+)(\s*(?:[><=~]+)\s*)([0-9.]+)([A-Za-z]+)').values
PDBbind_df['Kd_or_Ki_or_IC50'] = PDBbind_df['Kd_or_Ki_or_IC50'].str.replace('IC', 'IC50')

# convert the '-logKd/Ki' and 'value' columns to floats
# errors='coerce' parameter: In case the strings in the columns contain non-numeric characters, they will be converted to NaN values. 
PDBbind_df['-logKd/Ki'] = pd.to_numeric(PDBbind_df['-logKd/Ki'], errors='coerce')
PDBbind_df['value'] = pd.to_numeric(PDBbind_df['value'], errors='coerce')

PDBbind_df.head()

,PDB_ID,resolution,release_year,ligand_name,comment,Uniprot_ID,protein_name,-logKd/Ki,Kd/Ki,Kd_or_Ki_or_IC50,relation,value,unit
0,2TPI,2.10,1982,2-mer,,P00760,TRYPSINOGEN,4.31,Kd=49uM,Kd,=,49.00,uM
1,5TLN,2.30,1982,BAN,incomplete ligand structure,P00800,THERMOLYSIN,6.37,Ki=0.43uM,Ki,=,0.43,uM
2,4TLN,2.30,1982,LNO,,P00800,THERMOLYSIN,3.72,Ki=190uM,Ki,=,190.00,uM
3,4CTS,2.90,1984,OAA,,P00889,CITRATE SYNTHASE,5.00,Kd<10uM,Kd,<,10.00,uM
4,6RSA,NMR,1986,UVC,,P61823,RIBONUCLEASE A,4.40,Ki=40uM,Ki,=,40.00,uM


In [13]:
print(PDBbind_df['unit'].value_counts()) 

unit
uM    10342
nM     7862
mM      908
pM      223
fM       11
Name: count, dtype: int64


In [14]:
# Convert the values and unit all to standard_values 

# Define a mapping dictionary for the units
unit_conversion_factors = { 'mM': 1e6, 
                            'uM': 1e3, 
                            'nM': 1, 
                            'pM': 1e-3, 
                            'fM': 1e-6  }

# Define a function to convert the units
def convert_units(row):
    unit = row['unit']
    value = row['value']
    conversion_factor = unit_conversion_factors[unit]
    standard_value = value * conversion_factor
    return standard_value

# Apply the function to each row and create a new column for the standard_value
PDBbind_df['standard_value'] = PDBbind_df.apply(convert_units, axis=1)
PDBbind_df['standard_unit'] = 'nM'

# Remove any trailing zeros
PDBbind_df['value'] = PDBbind_df['value'].astype(str).str.rstrip('0').str.rstrip('.')
PDBbind_df['standard_value'] = PDBbind_df['standard_value'].astype(str).str.rstrip('0').str.rstrip('.')

PDBbind_df.head()

,PDB_ID,resolution,release_year,ligand_name,comment,Uniprot_ID,protein_name,-logKd/Ki,Kd/Ki,Kd_or_Ki_or_IC50,relation,value,unit,standard_value,standard_unit
0,2TPI,2.10,1982,2-mer,,P00760,TRYPSINOGEN,4.31,Kd=49uM,Kd,=,49,uM,49000,nM
1,5TLN,2.30,1982,BAN,incomplete ligand structure,P00800,THERMOLYSIN,6.37,Ki=0.43uM,Ki,=,0.43,uM,430,nM
2,4TLN,2.30,1982,LNO,,P00800,THERMOLYSIN,3.72,Ki=190uM,Ki,=,190,uM,190000,nM
3,4CTS,2.90,1984,OAA,,P00889,CITRATE SYNTHASE,5.00,Kd<10uM,Kd,<,10,uM,10000,nM
4,6RSA,NMR,1986,UVC,,P61823,RIBONUCLEASE A,4.40,Ki=40uM,Ki,=,40,uM,40000,nM


In [15]:
# drop redundant columns
#PDBbind_df.drop(['Kd/Ki', 'value', 'unit'], axis=1, inplace=True)

In [16]:
# Write table to .csv 
PDBbind_df.to_csv('PDBbind_V2020.csv', index=False, header=True, sep='|')

In [17]:
# Display a specific row based on the PDB_ID
row = PDBbind_df[PDBbind_df['PDB_ID'].isin(['1RE8'])]
row

,PDB_ID,resolution,release_year,ligand_name,comment,Uniprot_ID,protein_name,-logKd/Ki,Kd/Ki,Kd_or_Ki_or_IC50,relation,value,unit,standard_value,standard_unit
2376,1RE8,2.10,2004,BD2,,P05132,CAMP-DEPENDENT PROTEIN KINASE,9.52,Ki=0.3nM,Ki,=,0.3,nM,0.3,nM


In [18]:
# # define the list of Uniprot IDs to keep
# PKA_UniProt = ['P17612', 'P05132']

# # keep only rows for which the 'Uniprot_ID' is in the list named 'PKA_UniProt'
# PDBbind_df_filtered = PDBbind_df[PDBbind_df['Uniprot_ID'].isin(PKA_UniProt)]
# PDBbind_df_filtered.reset_index(inplace=True, drop=True)
# PDBbind_df_filtered

## Binding Data for the PKA structures under investigation 
(as obtained from the 'ChemSpaceAnalysis_PDB.ipynb' JupyterNotebook)